<a href="http://cocl.us/pytorch_link_top?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0321ENSkillsNetwork20647850-2021-01-01">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/Pytochtop.png" width="750" alt="IBM Product " />
</a> 


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/cc-logo-square.png" width="200" alt="cognitiveclass.ai logo" />


<h1>Objective</h1><ul><li> How to use linear classifier in pytorch.</li></ul> 


<h1>Linear  Classifier with PyTorch </h1>


<p>Before you use a  Deep neural network to solve the classification problem,  it 's a good idea to try and solve the problem with the simplest method. You will need the dataset object from the previous section.
In this lab, we solve the problem with a linear classifier.
 You will be asked to determine the maximum accuracy your linear classifier can achieve on the validation data for 5 epochs. We will give some free parameter values if you follow the instructions you will be able to answer the quiz. Just like the other labs there are several steps, but in this lab you will only be quizzed on the final result. </p>


<h2>Table of Contents</h2>


<div class="alert alert-block alert-info" style="margin-top: 20px">

<ul>
    <li><a href="https://#download_data"> Download data</a></li>
    <li><a href="https://#auxiliary"> Imports and Auxiliary Functions </a></li>
    <li><a href="https://#data_class"> Dataset Class</a></li>
    <li><a href="https://#trasform_Data_object">Transform Object and Dataset Object</a></li>
    <li><a href="https://#Question">Question</a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>
 </div>
<hr>


<h2 id="download_data">Download Data</h2>


In this section, you are going to download the data from IBM object storage using <b>wget</b>, then unzip them.  <b>wget</b> is a command the retrieves content from web servers, in this case its a zip file. Locally we store the data in the directory  <b>/resources/data</b> . The <b>-p</b> creates the entire directory tree up to the given directory.


First, we download the file that contains the images, if you dint do this in your first lab uncomment:


In [2]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip -P /resources/data

--2021-12-16 09:48:45--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘/resources/data/concrete_crack_images_for_classification.zip’

concrete_crack_imag 100%[===================>] 233.90M  31.6MB/s    in 6.6s    

2021-12-16 09:48:52 (35.6 MB/s) - ‘/resources/data/concrete_crack_images_for_classification.zip’ saved [245259777/245259777]



We then unzip the file, this ma take a while:


In [3]:
!unzip -q  /resources/data/concrete_crack_images_for_classification.zip -d  /resources/data

We then download the files that contain the negative images:


<h2 id="auxiliary">Imports and Auxiliary Functions</h2>


The following are the libraries we are going to use for this lab:


In [4]:
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim 

<h2 id="data_class">Dataset Class</h2>


In this section, we will use the previous code to build a dataset class. As before, make sure the even samples are positive, and the odd samples are negative.  If the parameter <code>train</code> is set to <code>True</code>, use the first 30 000  samples as training data; otherwise, the remaining samples will be used as validation data. Do not forget to sort your files so they are in the same order.


In [5]:
class Dataset(Dataset):

    # Constructor
    def __init__(self,transform=None,train=True, is_flat=True):
        directory="/resources/data"
        positive="Positive"
        negative="Negative"

        positive_file_path=os.path.join(directory,positive)
        negative_file_path=os.path.join(directory,negative)
        positive_files=[os.path.join(positive_file_path,file) for file in  os.listdir(positive_file_path) if file.endswith(".jpg")]
        positive_files.sort()
        negative_files=[os.path.join(negative_file_path,file) for file in  os.listdir(negative_file_path) if file.endswith(".jpg")]
        negative_files.sort()
        number_of_samples=len(positive_files)+len(negative_files)
        self.all_files=[None]*number_of_samples
        self.all_files[::2]=positive_files
        self.all_files[1::2]=negative_files 
        # The transform is goint to be used on image
        self.transform = transform
        #torch.LongTensor
        self.Y=torch.zeros([number_of_samples]).type(torch.LongTensor)
        self.Y[::2]=1
        self.Y[1::2]=0
        
        if train:
            self.all_files=self.all_files[0:30000]
            self.Y=self.Y[0:30000]
            self.len=len(self.all_files)
        else:
            self.all_files=self.all_files[30000:]
            self.Y=self.Y[30000:]
            self.len=len(self.all_files)    
       
    # Get the length
    def __len__(self):
        return self.len
    
    # Getter
    def __getitem__(self, idx):
        
        
        image=Image.open(self.all_files[idx])
        y=self.Y[idx]
          
        
        # If there is any transform method, apply it onto the image
        if self.transform:
            image = self.transform(image)
        return image, y

<h2 id="trasform_Data_object">Transform Object and Dataset Object</h2>


Create a transform object, that uses the <code>Compose</code> function. First use the transform <code>ToTensor()</code> and followed by <code>Normalize(mean, std)</code>. The value for <code> mean</code> and <code>std</code> are provided for you.


In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# transforms.ToTensor()
#transforms.Normalize(mean, std)
#transforms.Compose([])

transform_flatten =transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean, std), transforms.Lambda(lambda x: torch.flatten(x))])
transform = transforms.Compose([ transforms.ToTensor(), transforms.Resize((224,224),interpolation=2), transforms.Normalize(mean, std)])

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Create object for the training data  <code>dataset_train</code> and validation <code>dataset_val</code>. Use the transform object to convert the images to tensors using the transform object:


In [50]:
dataset_train=Dataset(transform=transform_flatten,train=True)
dataset_val=Dataset(transform=transform_flatten,train=False)

We  can find the shape of the image:


In [13]:
dataset_train[0][0].shape

torch.Size([154587])

We see that it's a color image with three channels:


In [14]:
size_of_image=3*227*227
size_of_image

154587

<h2 id="Question"> Question <h2>


<b> Create a custom module for Softmax for two classes,called model. The input size should be the <code>size_of_image</code>, you should record the maximum accuracy achieved on the validation data for the different epochs. For example if the 5 epochs the accuracy was 0.5, 0.2, 0.64,0.77, 0.66 you would select 0.77.</b>


Train the model with the following free parameter values:


<b>Parameter Values</b>

   <li>learning rate:0.1 </li>
   <li>momentum term:0.1 </li>
   <li>batch size training:1000</li>
   <li>Loss function:Cross Entropy Loss </li>
   <li>epochs:5</li>
   <li>set: torch.manual_seed(0)</li>


In [15]:
torch.manual_seed(0)

<b>Custom Module:</b>


In [16]:
import torch.nn.functional as F

class LinearClassifier(nn.Module):
  def __init__(self, input_size, output_size):
    super(LinearClassifier, self).__init__()
    self.linear = nn.Linear(input_size, output_size)
    
  def forward(self, x):
    out = self.linear(x)
    out = F.softmax(out, dim=0)
    return out

<b>Model Object:</b>


In [24]:
model = LinearClassifier(size_of_image, 2).to('cuda')

<b>Optimizer:</b>


In [25]:
from torch import optim
optimizer = optim.SGD(model.parameters(), lr=0.1) 

<b>Criterion:</b>


In [26]:
criterion = nn.CrossEntropyLoss()

<b>Data Loader Training and Validation:</b>


In [27]:
train_loader = DataLoader(dataset=dataset_train, batch_size=1000)
val_loader = DataLoader(dataset=dataset_val, batch_size=1000)

In [28]:
len(dataset_val)

10000

In [29]:
len(val_loader)

10

<b>Train Model with 5 epochs, should take 35 minutes: </b>


In [31]:
for epoch in range(5):
  for x, y in train_loader:
    x, y = x.to('cuda'), y.to('cuda')
    yhat = model(x)
    loss = criterion(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  num_acc = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to('cuda'), y.to('cuda')
      yhat = model(x)
      pred = torch.argmax(yhat, dim=1)
      num_acc += torch.sum(pred == y)
  print('epoch {} acc : {}'.format(epoch, num_acc/len(dataset_val)))

epoch 0 acc : 0.6822999715805054
epoch 1 acc : 0.6836999654769897
epoch 2 acc : 0.6850999593734741
epoch 3 acc : 0.6857999563217163
epoch 4 acc : 0.6877999901771545


In [ ]:
torch.argmax(torch.tensor([[0.3, 0.8],[0.7, 0.4]]), dim=1)

In [ ]:
torch.sum(torch.tensor([0,1,1]) == torch.tensor([1,1,1]))

In [17]:
torch.tensor([[0.3, 0.8],[0.7, 0.4]]) > 0.7

tensor([[False,  True],
        [False, False]])

# Use Resnet50 Model

In [7]:
torch.cuda.empty_cache()

In [8]:
dataset_train=Dataset(transform=transform,train=True)
dataset_val=Dataset(transform=transform,train=False)

In [9]:
dataset_train[0][0].shape

torch.Size([3, 224, 224])

In [23]:
train_loader = DataLoader(dataset=dataset_train, batch_size=1000)
val_loader = DataLoader(dataset=dataset_val, batch_size=1000)

In [11]:
from torchvision import models

In [12]:
resnet18 = models.resnet18(pretrained=True).to('cuda')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [22]:
for param in resnet18.parameters():
  param.requires_grad = False

In [13]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
classifier = nn.Linear(1000, 2).to('cuda')

In [15]:
#model = nn.Sequential(resnet50, classifier)
#model = model.to('cuda')

In [16]:
from torch import optim
optimizer = optim.SGD(classifier.parameters(), lr=0.1) 

In [17]:
criterion = nn.CrossEntropyLoss()

In [25]:
for epoch in range(5):
  for x, y in train_loader:
    x, y = x.to('cuda'), y.to('cuda')
    feature = resnet18(x)
    yhat = classifier(feature)
    loss = criterion(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
  num_acc = 0
  with torch.no_grad():
    for x, y in val_loader:
      x, y = x.to('cuda'), y.to('cuda')
      feature = resnet18(x)
      yhat = classifier(feature)
      pred = torch.argmax(yhat, dim=1)
      num_acc += torch.sum(pred == y)
  print('epoch {} acc : {}'.format(epoch, num_acc/len(dataset_val)))

epoch 0 acc : 0.9891999959945679
epoch 1 acc : 0.9896999597549438
epoch 2 acc : 0.9898999929428101
epoch 3 acc : 0.990399956703186
epoch 4 acc : 0.9904999732971191


<h2>About the Authors:</h2>
 <a href=https://"https//www.linkedin.com/in/joseph-s-50398b136//%22?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0321ENSkillsNetwork20647850-2021-01-01>Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                          |
| ----------------- | ------- | ---------- | ----------------------------------------------------------- |
| 2020-09-18        | 2.0     | Shubham    | Migrated Lab to Markdown and added to course repo in GitLab |


Copyright © 2019 <a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0321ENSkillsNetwork20647850-2021-01-01"> cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0321ENSkillsNetwork20647850-2021-01-01">MIT License</a>
